# 🚀 Welcome to the Hugging Face Uploader Notebook!

This notebook provides a user-friendly tool for uploading files directly to your Hugging Face repositories. Here's how to get started:

**🔑 Initial Setup (One Time Only)**

1.  **Authenticate:**
    *   Run the `notebook_login()` cell *once* to securely store your Hugging Face API token.
    *   **Important:** For security, avoid sharing your notebook file or system state after you have run `notebook_login()`. Do not commit your notebook file to a shared repo, as this could expose your API token.

**🗂️ Using the Uploader**

1.  **Repository Details:**
    *   Enter your Hugging Face Organization or Username in the "Owner" field.
    *   Enter your repository name in the "Repo" field.

2.  **Directory Selection:**
    *  Enter a directory path where your files are located.
    *  Select the 'Update Dir' button to set that path.

3.  **File Selection:**
    *   Select the appropriate file type from the dropdown menu.
    *   Select the files you want to upload from the list. You can sort them by name or date modified.

4.  **Commit Message (Optional):**
    *   Add a commit message to your upload, or use the default message.

5.  **Upload Options:**
    *   Choose whether to create a pull request or upload directly to the main branch.
    *   Select whether to clear the output after a successful upload.

6.  **Start Upload:**
    *   Click the "⬆️ Upload" button.

**💡 Important Notes**

*   **Direct Uploads:** This uploader uses the Hugging Face API for direct file uploads, bypassing the need for traditional Git operations for core functionality.
*   **Git LFS:** Most users will not need to interact with Git or Git LFS. If you need to clone or push changes to a repository *outside this notebook*, you will require a separate Git credential setup (e.g., your operating system's credential manager or SSH keys).  This is separate from your Hugging Face API token, and you should not share any git credentials in your notebook.
*   **Troubleshooting:** If you encounter any issues, please review the steps, or double-check that you have write access to the repository, and that your API token has the correct scope of access.

**📣 Updates & Community**

*   This tool will continue to be updated.
*   For the latest patches, fixes, and community contributions, visit [https://github.com/duskfallcrew/HuggingFace_Backup](https://github.com/duskfallcrew/HuggingFace_Backup)

We hope this notebook makes your Hugging Face uploads easier! If you have any questions or suggestions, please reach out.

In [2]:
!pip install -q huggingface_hub ipywidgets tqdm

# ✨ Connecting to Hugging Face: Authentication Required

To upload files to Hugging Face using this notebook, you'll need to authenticate using your Hugging Face API token. This token acts as your secure access key to the Hugging Face Hub.

**🔑 Obtaining Your API Token**

1.  **Go to Your Settings:** Navigate to your [Hugging Face account settings](https://huggingface.co/settings/tokens).
2.  **Access Tokens Section:** Find the "Access Tokens" section.
3.  **Create or Copy Token:**
    *   If you don't have one already, create a new access token.
    *   Copy your existing access token.

**🔒 Logging In**

1.  **Run the Cell:** Execute the cell below this section.
2.  **Paste Token:** When prompted, paste your Hugging Face API token into the input field.
3.  **Confirm:** Press `Enter` to complete the login process. No characters will be displayed as you type your token.
    *   **Security Note:** This ensures your token is kept private while pasting.

**Important Notes**

*   **Purpose:** This login step authorizes your notebook environment to interact with the Hugging Face Hub. This is required for uploading your models and data.
*   **Session-Based:** You will need to perform this login procedure each time you restart the notebook or start a new session (the token is stored locally on your computer, but not in the notebook, and it doesn't persist between sessions).
*  **Security:** After you have logged in using `notebook_login()`, **do not** share your notebook or your system state with untrusted users, and do not commit your notebook to a shared repository, as this could compromise your API token.

After completing this step, you'll be ready to use the upload functionality of this notebook.

In [3]:
from huggingface_hub import notebook_login
import os
notebook_login()

# 🚀 Using the Hugging Face File Uploader

To upload files to your Hugging Face repository, please follow these steps:

**Before you start**: You **must have already completed the following:**

  * You have authenticated with Hugging Face by running the `notebook_login()` cell located at the start of the notebook.
  * You have created your repository on the Hugging Face Hub.

**Using the Uploader**

1.  **Repository Details:**
    *   Enter your Hugging Face Organization or Username in the "Owner" field.
    *   Enter your repository name in the "Repo" field.

2.  **Directory Selection**
     *  Enter a directory path where your files are located.
     *  Select the 'Update Dir' button to set that path.

3.  **File Selection:**
    *   Select the appropriate file type from the dropdown menu (e.g., SafeTensors, PyTorch).
    *   Choose how you want to sort your files (by name or date modified).
    *   Select the files you wish to upload from the list.

4.  **Upload Settings:**
    *   Enable "Create Pull Request" if you want to submit your changes through a pull request, otherwise the changes will be made directly to the main branch of your repo.
    *   Click the "Upload" button to start the upload process.

**Status and Updates**

*   The uploader will provide progress messages during the upload process.
*   Any errors will be reported with suggestions on how to fix them.

**Reusing the Uploader**

*   You can repeat these steps to upload additional files to the same repository.

In [20]:
import glob
import os
from pathlib import Path
from huggingface_hub import HfApi
from ipywidgets import Text, Dropdown, Button, SelectMultiple, VBox, HBox, Output, Layout, Checkbox, HTML, Textarea, Label, IntProgress
from IPython.display import display, clear_output
import time
from tqdm import tqdm

class HuggingFaceUploader:
    def __init__(self):
        self.api = HfApi()
        self.file_types = [
            # AI Model Files 🤖
            ('SafeTensors', 'safetensors'),
            ('PyTorch Models', 'pt'),
            ('PyTorch Legacy', 'pth'),
            ('ONNX Models', 'onnx'),
            ('TensorFlow Models', 'pb'),
            ('Keras Models', 'h5'),

            # Checkpoint Files 🎯
            ('Checkpoints', 'ckpt'),
            ('Binary Files', 'bin'),

            # Config & Data Files 📝
            ('JSON Files', 'json'),
            ('YAML Files', 'yaml'),
            ('YAML Alt', 'yml'),
            ('Text Files', 'txt'),
            ('CSV Files', 'csv'),
            ('Pickle Files', 'pkl'),

            # Image Files 🎨
            ('PNG Images', 'png'),
            ('JPEG Images', 'jpg'),
            ('JPEG Alt', 'jpeg'),
            ('WebP Images', 'webp'),
            ('GIF Images', 'gif'),

            # Archive Files 📦
            ('ZIP Archives', 'zip'),
            ('TAR Files', 'tar'),
            ('GZ Archives', 'gz')
        ]
        self.current_directory = os.getcwd()
        self._create_widgets()
        self._bind_events()


    def _create_widgets(self):
        # Repository info section
        self.repo_info = HTML(value="<b>📚 Repository Details</b>")
        self.org_name = Text(
            placeholder='Organization or Username',
            description='Owner:',
            style={'description_width': 'initial'}
        )
        self.repo_name = Text(
            placeholder='Repository Name',
            description='Repo:',
            style={'description_width': 'initial'}
        )

        # File handling section
        self.file_section = HTML(value="<b>🗂️ File Selection</b>")
        self.file_type = Dropdown(
            options=self.file_types,
            value='safetensors',
            description='File Type:',
            style={'description_width': 'initial'}
        )
        self.sort_by = Dropdown(
            options=['name', 'date'],
            value='name',
            description='Sort By:'
        )
        self.directory_label = Label(value=f"Current Directory: {self.current_directory}")
        self.directory_text = Text(
             value=self.current_directory,
            description="Path:",
            style={'description_width': 'initial'},
            layout=Layout(width="400px")
        )
        self.directory_update_btn = Button(
            description='🔄 Update Dir',
            button_style='info',
            tooltip='Refresh directory'
        )


        # Custom commit message
        self.commit_section = HTML(value="<b>💭 Commit Details</b>")
        self.commit_msg = Textarea(
            value="Upload with Earth & Dusk Huggingface 🤗 Backup",
            placeholder='Enter your commit message (optional)',
            description='Message:',
            layout=Layout(width='400px', height='60px')
        )

        # Upload options section
        self.upload_section = HTML(value="<b>🚀 Upload Settings</b>")
        self.create_pr = Checkbox(
            value=False,
            description='Create Pull Request',
            indent=False,
            tooltip='When checked, creates a Pull Request instead of direct upload'
        )
        self.clear_after = Checkbox(
            value=True,
            description='Clear output after upload',
            indent=False,
            tooltip='Clears output area after successful upload'
        )

        # Progress tracking
        self.progress_text = HTML(value="")

        # Action buttons with style
        self.update_btn = Button(
            description='🔄 Update Files',
            button_style='info',
            tooltip='Refresh file list'
        )
        self.upload_btn = Button(
            description='⬆️ Upload',
            button_style='success',
            tooltip='Start upload process',
             layout=Layout(width='auto', height='auto'),
        )
        self.clear_btn = Button(
            description='🧹 Clear Output',
            button_style='warning',
            tooltip='Clear output area'
        )

        # File selector and output
        self.ckpt_picker = SelectMultiple(
            options=[],
            layout=Layout(width="600px", height="200px")
        )
        self.out = Output(layout=Layout(padding='10px'))
        
         # Create debug output:
        self.debug_output = Output()
        self.debug_box = VBox([HTML(value="<b style='color: #2ecc71;'>Debug Output</b>"), self.debug_output],
                           layout=Layout(border='2px solid #2ecc71', padding='10px', margin_top='10px'))
        self.debug_toggle = Checkbox(
            value=False,
            description='Show Debug',
            indent=False,
            tooltip='Show debug information'
        )
        self.debug_box.layout.visibility = 'hidden'


    def _bind_events(self):
        self.update_btn.on_click(self._update_files)
        self.upload_btn.on_click(self._upload_ckpts)
        self.clear_btn.on_click(lambda _: self.out.clear_output())
        self.file_type.observe(self._update_files, names='value')
        self.directory_update_btn.on_click(self._update_directory)
        self.debug_toggle.observe(self._toggle_debug, names='value')

    def _toggle_debug(self,_):
      if self.debug_toggle.value == True:
        self.debug_box.layout.visibility = 'visible'
      else:
        self.debug_box.layout.visibility = 'hidden'


    def _update_directory(self,_):
        new_dir = self.directory_text.value
        if os.path.isdir(new_dir):
            self.current_directory = new_dir
            self.directory_label.value = f"Current Directory: {self.current_directory}"
            self._update_files(None)
        else:
            with self.out:
              print("❌ Invalid Directory")

    def _update_files(self, _):
         file_extension = self.file_type.value
         try:
            # Glob files based on file extension
            all_files = glob.glob(os.path.join(self.current_directory, f"*.{file_extension}"))

            with self.debug_output:
              print(f"DEBUG _update_files: File type is {file_extension}")
              print(f"DEBUG _update_files: All files found by glob: {all_files}")

            # Filter out symlinks, old files and ignore patterns
            filtered_files = []
            for file_path in all_files:
              if os.path.islink(file_path):
                 with self.debug_output:
                    print(f"DEBUG _update_files: Skipping symlink {file_path}")
                 continue
              if not os.path.isfile(file_path):
                  with self.debug_output:
                     print(f"DEBUG _update_files: Skipping non file {file_path}")
                  continue

              filtered_files.append(file_path)
            
            # Sort the files
            all_ckpts = sorted(
                filtered_files,
                 key=os.path.getmtime if self.sort_by.value == 'date' else str
            )

            self.ckpt_picker.options = all_ckpts

            with self.out:
                 print(f"✨ Found {len(all_ckpts)} {file_extension} files in {self.current_directory}")

         except Exception as e:
            with self.out:
                print(f"❌ Error listing files: {str(e)}")


    def _format_size(self, size):
        for unit in ['B', 'KB', 'MB', 'GB']:
            if size < 1024:
                return f"{size:.2f} {unit}"
            size /= 1024
        return f"{size:.2f} TB"

    def _print_file_info(self, file_path, file_size, index, total):
         with self.out:
                print(f"📦 File {index}/{total}: {file_path} ({self._format_size(file_size)})")


    def _upload_ckpts(self, _):
        if not self.org_name.value or not self.repo_name.value:
            with self.out:
                print("❗ Please fill in both Organization/Username and Repository name")
                return

        repo_id = f"{self.org_name.value}/{self.repo_name.value}"
        selected_files = self.ckpt_picker.value
        total_files = len(selected_files)

        with self.out:
            if not selected_files:
                print("📝 Nothing selected for upload. Please select files from the list.")
                return

            print(f"🎯 Starting upload to: huggingface.co/{repo_id}")

        commit_msg = self.commit_msg.value if self.commit_msg.value else "Uploaded with Earth & Dusk Huggingface 🤗 Backup"

        with self.debug_output:
            print(f"DEBUG TQDM: Is TQDM working? This should be above the progress bar.")

        progress_bar = IntProgress( # Create a progress bar for this upload
            value=0,
            min=0,
            max=total_files,
            description='Upload Progress:',
            style={'description_width': 'initial'},
            layout=Layout(width="400px"),
          )
        display(progress_bar)  # Display progress bar

        for idx, ckpt in enumerate(selected_files, 1):
            try:
                file_size = os.path.getsize(ckpt)
                self._print_file_info(ckpt, file_size, idx, total_files)
                start_time = time.time()
                with self.debug_output:
                   print(f"🚀 Attempting upload of {ckpt}")
                response = self.api.upload_file(
                    path_or_fileobj=ckpt,
                    path_in_repo=os.path.basename(ckpt),
                    repo_id=repo_id,
                    repo_type=None,
                    create_pr=self.create_pr.value,
                    commit_message=commit_msg
                )
                with self.debug_output:
                    print(f"✅ Upload Response: {response}")
                duration = time.time() - start_time
                with self.out:
                   print(f"✅ Upload completed in {duration:.1f} seconds")
                   display(response)


                progress_bar.value = idx
                display(progress_bar) # Force a render of the progress bar
            except Exception as e:
                 with self.out:
                    print(f"❌ Error uploading {ckpt}: {str(e)}")
                 continue


        with self.out:
            print("\n✨ All uploads completed! ✨")
            if self.create_pr.value:
                print("🎉 Check your repository for the new Pull Request!")
            else:
                print("🎉 Files have been uploaded directly to your repository!")

            if self.clear_after.value:
                time.sleep(3)
                self.out.clear_output()

    def display(self):
        box = VBox([
            self.repo_info,
            HBox([self.org_name, self.repo_name]),
            self.file_section,
            HBox([self.file_type, self.sort_by]),
            HBox([self.directory_label, self.directory_text, self.directory_update_btn]),
            self.commit_section,
            self.commit_msg,
            self.upload_section,
            HBox([self.create_pr, self.clear_after]),
            self.update_btn,
            self.ckpt_picker,
            HBox([self.upload_btn, self.clear_btn, self.debug_toggle], layout=Layout(align_items='center')),
             self.progress_text,
            self.out,
            self.debug_box,
        ])
        display(box)

# Create and display the uploader - just one line to rule them all! ✨
uploader = HuggingFaceUploader()
uploader.display()